#Data Preprocessing

### Remove empty rows

In [22]:
import pandas as pd
df = pd.read_csv('train.csv')

df.head()

,Unnamed: 0,Time,data,label
0,0,[00:12.55,Another day wasted out of time,Angry
1,1,[00:14.69,I can't get out of this,Angry
2,2,[00:16.03,Altered state of mind,Angry
3,3,[00:17.41,I'm going overboard,Angry
4,4,[00:18.75,My conscience meets decline,Angry


In [23]:
len(df)

20115

In [ ]:
df.dropna(
    axis=0,
    how='any',
    thresh=None,
    subset=None,
    inplace=True
)

In [ ]:
len(df)

17674

### Label Encoder

In [ ]:
from sklearn.preprocessing import LabelEncoder
import random
import pickle
import numpy as np

X_train = df['data'].values 

y_train = df['label'].values

print('before: %s ...' %y_train[:5])

le = LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)

print('after: %s ...' %y_train[:5])

before: ['Angry' 'Angry' 'Angry' 'Angry' 'Angry'] ...
after: [0 0 0 0 0] ...


### Porter Stemmer

In [ ]:
import nltk
import string
import re

porter_stemmer = nltk.stem.porter.PorterStemmer()

def porter_tokenizer(text, stemmer=porter_stemmer):
    """
    A Porter-Stemmer-Tokenizer hybrid to splits sentences into words (tokens) 
    and applies the porter stemming algorithm to each of the obtained token. 
    Tokens that are only consisting of punctuation characters are removed as well.
    Only tokens that consist of more than one letter are being kept.
    
    Parameters
    ----------
        
    text : `str`. 
      A sentence that is to split into words.
        
    Returns
    ----------
    
    no_punct : `str`. 
      A list of tokens after stemming and removing Sentence punctuation patterns.
    
    """
    lower_txt = text.lower()
    tokens = nltk.wordpunct_tokenize(lower_txt)
    stems = [porter_stemmer.stem(t) for t in tokens]
    no_punct = [s for s in stems if re.match('^[a-zA-Z]+$', s) is not None]
    return no_punct

In [ ]:
porter_tokenizer("Don't !!! --- want swimming. ")

['don', 't', 'want', 'swim']

### Stop words

In [ ]:
with open('stopwords.txt', 'r') as infile:
    stop_words = infile.read().splitlines()
print('stop words %s ...' %stop_words[:5])

stop words ["a's", 'able', 'about', 'above', 'according'] ...


###Count Vectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(
            encoding='utf-8',
            decode_error='replace',
            strip_accents='unicode',
            analyzer='word',
            binary=False,
            stop_words=stop_words,
            tokenizer=porter_tokenizer,
            ngram_range=(1,1)
    )

In [ ]:
vocab = ["123 1 The\n swimmer likes swimming so he swims. Don't didn`t"]

vec = vec.fit(vocab)

sentence1 = vec.transform([u'The swimmer likes swimming.'])
sentence2 = vec.transform(['The\nswimmer \nswims.'])


print('TEST:')
print('Vocabulary: %s' %vec.get_feature_names())
print('Sentence 1: %s' %sentence1.toarray())
print('Sentence 2: %s' %sentence2.toarray())

TEST:
Vocabulary: ['didn', 'don', 'swim', 'swimmer', 't']
Sentence 1: [[0 0 1 1 0]]
Sentence 2: [[0 0 1 1 0]]


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"
/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['a', 'abl', 'abov', 'accord', 'accordingli', 'actual', 'afterward', 'ain', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anybodi', 'anyon', 'anyth', 'anywher', 'appreci', 'appropri', 'aren', 'asid', 'associ', 'avail', 'aw', 'becam', 'becaus', 'becom', 'befor', 'believ', 'besid', 'c', 'caus', 'certainli', 'chang', 'clearli', 'concern', 'consequ', 'consid', 'correspond', 'couldn', 'cours', 'current', 'd', 'definit', 'describ', 'despit', 'didn', 'differ', 'doe', 'doesn', 'don', 'downward', 'dure', 'els', 'elsewher', 'entir', 'especi', 'everi', 'everybodi', 'everyon', 'everyth', '

In [ ]:
X_train = df['data'].values 
vec = vec.fit(X_train.ravel())
print('Vocabulary size: %s' %len(vec.get_feature_names()))

Vocabulary size: 4333


In [ ]:
vec = CountVectorizer(
            encoding='utf-8',
            decode_error='replace',
            strip_accents='unicode',
            analyzer='word',
            binary=False,
            stop_words=stop_words,
            tokenizer=porter_tokenizer,
            ngram_range=(2,2)
    )
# N-grams = 2
vocab = ["123 1 The\n swimmer likes swimming so he swims. Don't didn`t"]

vec = vec.fit(vocab)

sentence1 = vec.transform([u'The swimmer likes swimming.'])
sentence2 = vec.transform(['The\nswimmer \nswims.'])


print('TEST:')
print('Vocabulary: %s' %vec.get_feature_names())
print('Sentence 1: %s' %sentence1.toarray())
print('Sentence 2: %s' %sentence2.toarray())

TEST:
Vocabulary: ['didn t', 'don t', 'swim don', 'swim swim', 'swimmer swim', 't didn']
Sentence 1: [[0 0 0 0 1 0]]
Sentence 2: [[0 0 0 0 1 0]]


### Tfidf Vectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
            encoding='utf-8',
            decode_error='replace',
            strip_accents='unicode',
            analyzer='word',
            binary=False,
            stop_words=stop_words,
            tokenizer=porter_tokenizer
    )

In [ ]:
vocab = ["123 1 The\n swimmer likes swimming so he swims. Don't didn`t"]

tfidf = tfidf.fit(vocab)

sentence1 = tfidf.transform([u'The swimmer likes swimming.'])
sentence2 = tfidf.transform(['The\nswimmer \nswims.'])


print('TEST:')
print('Vocabulary: %s' %tfidf.get_feature_names())
print('Sentence 1: %s' %sentence1.toarray())
print('Sentence 2: %s' %sentence2.toarray())

TEST:
Vocabulary: ['didn', 'don', 'swim', 'swimmer', 't']
Sentence 1: [[0.         0.         0.70710678 0.70710678 0.        ]]
Sentence 2: [[0.         0.         0.70710678 0.70710678 0.        ]]


In [ ]:
tfidf = tfidf.fit(X_train.ravel())

print('Vocabulary size: %s' %len(tfidf.get_feature_names()))

Vocabulary size: 4333
